# Refactotron: Maximum Optimized LoRA Training (Colab Pro)

**Colab Pro Version - Option C: Balanced & Powerful**

**Key Features:**
- ✅ Checkpoints auto-save to Google Drive (survives disconnects)
- ✅ max_length=1536 (97% coverage vs 90% at 1024)
- ✅ LoRA rank=24 (50% more capacity than r=16)
- ✅ 100% syntactically valid training data (39,812 samples)
- ✅ Learning rate: 2e-5 (optimized for fine-tuning)
- ✅ Cosine LR scheduler with 500-step warmup
- ✅ Moderate regularization (Option B)

**Expected Results:**
- Validation Loss: 0.45-0.50 (vs 0.68 before)
- Training time: ~15-18 hours on T4 GPU
- BLEU: 75-80 (target: 73.5)
- CodeBERT: 0.88-0.92 (target: 0.87)

## Cell 1: Mount Google Drive (CRITICAL - Run First!)

This ensures all checkpoints save to YOUR Google Drive permanently.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Save checkpoints in drive
print("/content/drive/MyDrive/refactotron_lora_optimized/")


## Cell 2: Check GPU & Install Dependencies

In [ ]:
# Check if GPU is available and install dependencies
import torch

print("GPU Status:")
print(f"Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("Ready for training!")
else:
    print("   ⚠️  NO GPU! Go to: Runtime > Change runtime type > T4 GPU")

!pip install -q transformers datasets peft accelerate bitsandbytes

## Cell 3: Upload Training Data

**Upload these 2 files:**
- `train_enhanced.jsonl` (60.2 MB)
- `validation_enhanced.jsonl` (7.5 MB)

Click the folder icon 📁 on left sidebar and drag files, OR run the cell below:

In [ ]:
# Upload training data
from google.colab import files
import os

print("Upload train_enhanced.jsonl and validation_enhanced.jsonl")
uploaded = files.upload()

print("\nFiles uploaded:")
for filename in uploaded.keys():
    size_mb = len(uploaded[filename]) / (1024*1024)
    print(f"{filename}: {size_mb:.1f} MB")

## Cell 4: HuggingFace Authentication

In [ ]:
# Authenticate with huggingface to access StarCoder-1B model
from huggingface_hub import login

login()

## Cell 5: Load Model & Tokenizer

In [ ]:
# Load Model and Tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType
import torch

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoderbase-1b")
tokenizer.pad_token = tokenizer.eos_token

# Load model in fp16 to save memory
model = AutoModelForCausalLM.from_pretrained(
    "bigcode/starcoderbase-1b",
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

print(f"Base model loaded: {model.num_parameters():,} parameters")

## Cell 6: Configure LoRA (OPTION C - Enhanced)

**Improvements over baseline:**
- Rank increased: 16 → 24 (50% more capacity)
- Alpha: 48 (2x rank)
- Trainable params: ~15M (vs ~10M)

In [ ]:

lora_config = LoraConfig(
    r=16,                                          # REDUCED from 24 to save memory
    lora_alpha=32,                                 # 2x rank
    target_modules=["c_proj", "c_attn"],           # REDUCED from 3 to 2 modules to save memory
    lora_dropout=0.08,                             # Moderate dropout
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

print("\nLoRA configured for T4 GPU memory constraints (r=16)")

## Cell 7: Load Training Data

In [ ]:
# Load training Data
from datasets import Dataset
import json

def load_jsonl(filepath):
    """Load JSONL file"""
    data = []
    with open(filepath, 'r') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    return data


train_data = load_jsonl('train_enhanced.jsonl')
val_data = load_jsonl('validation_enhanced.jsonl')

print(f"Train: {len(train_data):,} samples")
print(f"Validation: {len(val_data):,} samples")

# Convert to HuggingFace Dataset
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

# Preview
print("\nSample input (first 200 chars):")
print(train_data[0]['input'][:200])

## Cell 8: Tokenization (OPTION C - Extended Context)

**max_length=1536 captures ~97% of all training data (vs 90% at 1024)**

In [ ]:
# tokenization with max_length-1024 to fit in T4 GPU memory
from transformers import DataCollatorForLanguageModeling

def tokenize_function(examples):
    """
    Tokenize input + output together.
    """
    # Combine input and output
    full_texts = [inp + "\n" + out for inp, out in zip(examples['input'], examples['output'])]

    # Tokenize with 1024 max length (reduced from 1536 for memory)
    result = tokenizer(
        full_texts,
        truncation=True,
        max_length=1024,  # REDUCED from 1536 to fit T4 GPU memory
        padding=False,
    )

    # Set labels
    result["labels"] = result["input_ids"].copy()

    return result


tokenized_train = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names,
    desc="Tokenizing train"
)

tokenized_val = val_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=val_dataset.column_names,
    desc="Tokenizing validation"
)

print(f"Tokenized train: {len(tokenized_train):,} samples")
print(f"Tokenized validation: {len(tokenized_val):,} samples")

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

## Cell 9: Training Configuration (OPTION C - Balanced)

**Key optimizations:**
- max_length: 1536 (97% data coverage)
- LoRA rank: 24 (50% more capacity)
- Gradient accumulation: 6 (adjusted for longer sequences)
- Learning rate: 2e-5 (10x lower than original)
- Cosine LR scheduler with 500-step warmup
- Weight decay: 0.02 (moderate)
- LoRA dropout: 0.08 (moderate)
- Label smoothing: 0.05 (moderate)

In [ ]:
# Training configuration - OPTIMIZED FOR T4 GPU MEMORY
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

training_args = TrainingArguments(
    # Output - SAVES TO GOOGLE DRIVE!
    output_dir="/content/drive/MyDrive/refactotron_lora_optimized",
    logging_dir="/content/drive/MyDrive/refactotron_lora_optimized/logs",

    # Training schedule
    num_train_epochs=5,

    # Batch size - OPTIMIZED FOR MEMORY
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,  # INCREASED from 6 to maintain effective batch size

    # Learning rate (OPTIMIZED)
    learning_rate=2e-5,              # 10x lower than original
    lr_scheduler_type="cosine",      # Smooth decay
    warmup_steps=500,                # Better stability

    # Regularization
    weight_decay=0.02,               # Moderate L2 regularization
    label_smoothing_factor=0.05,     # Moderate label smoothing
    max_grad_norm=1.0,               # Gradient clipping

    # Precision - FIXED: Changed from fp16 to bf16
    bf16=True,

    # Logging & evaluation
    logging_steps=50,
    eval_steps=500,
    save_steps=500,                  # Save checkpoint every 500 steps
    save_total_limit=3,              # Keep best 3 checkpoints
    eval_strategy="steps",

    # Best model selection
    load_best_model_at_end=True,
    metric_for_best_model="loss",

    # Memory optimization - CRITICAL FOR T4
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},  # More memory efficient
    optim="adamw_torch_fused",       # More memory efficient optimizer
    
    # Reporting
    report_to="none",
)

# Early stopping
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3  # Stop if no improvement for 3 evals (1500 steps)
)

print("Training configuration complete (T4 GPU optimized)")

## Cell 10: Initialize Trainer & Training Summary

In [ ]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    callbacks=[early_stopping]
)

# Training summary
total_steps = (len(tokenized_train) //
               (training_args.per_device_train_batch_size *
                training_args.gradient_accumulation_steps) *
               training_args.num_train_epochs)

print(f"Training samples: {len(tokenized_train):,}")
print(f"Validation samples: {len(tokenized_val):,}")
print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"Total epochs: {training_args.num_train_epochs}")
print(f"Max training steps: {total_steps:,}")
print(f"Evaluation every: {training_args.eval_steps} steps")
print(f"Checkpoint save every: {training_args.save_steps} steps")
print(f"\nCheckpoints save to:")
print(f"/content/drive/MyDrive/refactotron_lora_optimized/")
print(f"\n=== T4 GPU OPTIMIZED CONFIGURATION ===")
print(f"   • max_length: 1024 (optimized for T4 memory)")
print(f"   • LoRA rank: 16 (memory efficient)")
print(f"   • LoRA alpha: 32")
print(f"   • LoRA targets: c_proj, c_attn (2 modules)")
print(f"   • Trainable params: ~10M")
print(f"   • Gradient accumulation: 8")
print(f"   • Learning rate: 2e-5")
print(f"   • LR scheduler: cosine")
print(f"   • Warmup: 500 steps")
print(f"   • Weight decay: 0.02")
print(f"   • LoRA dropout: 0.08")
print(f"   • Label smoothing: 0.05")
print(f"   • Precision: bfloat16")
print(f"   • Optimizer: adamw_torch_fused (memory efficient)")
print(f"   • Gradient checkpointing: enabled (use_reentrant=False)")
print(f"\nEXPECTED RESULTS:")
print(f"   • Validation loss: 0.48-0.55")
print(f"   • BLEU score: 70-75")
print(f"   • CodeBERT similarity: 0.85-0.90")
print(f"   • Estimated time: 12-15 hours (T4 GPU)")

## Cell 11: START TRAINING 🚀

**This will take ~15-18 hours.**

**What to expect:**
- Validation loss starts ~0.71, should drop to ~0.45-0.50
- Training loss may fluctuate (regularization working!)
- Early stopping will halt when validation plateaus
- Checkpoints auto-save to Google Drive every 500 steps

**You can:**
- ✅ Close browser tab (Colab Pro allows background execution)
- ✅ Let laptop sleep
- ✅ Come back later to check progress

**Monitor progress:** Check back every few hours to see validation loss decreasing

In [ ]:
import time

print("Starting training...")
print(f"Start time: {time.strftime('%Y-%m-%d %H:%M:%S')}")

# START TRAINING
trainer.train()

print("Training complete")
print(f"End time: {time.strftime('%Y-%m-%d %H:%M:%S')}")

## Cell 12: Save Final Model

**Run this after training completes to save the best model to Drive.**

In [ ]:
# Save the LoRA adapter
model.save_pretrained("/content/drive/MyDrive/refactotron_lora_FINAL")
tokenizer.save_pretrained("/content/drive/MyDrive/refactotron_lora_FINAL")

print("Saved to: /content/drive/MyDrive/refactotron_lora_FINAL/")
print("\nNext steps:")
print("   1. Test model on test_enhanced.jsonl")
print("   2. Calculate BLEU and CodeBERT scores")
print("   3. Generate sample refactorings")
print("   4. Compare against vanilla StarCoder baseline")

## Cell 13: Download Model (Optional)

**If you want to download the model directly from Colab:**

In [ ]:
# Zip the final model
!cd /content/drive/MyDrive && zip -r refactotron_lora_FINAL.zip refactotron_lora_FINAL/

# Download
from google.colab import files
files.download('/content/drive/MyDrive/refactotron_lora_FINAL.zip')

print("Model downloaded! You can also access it anytime from Google Drive.")